In [1]:
from banners import PostgresBanner
wave_banner = PostgresBanner()
watch_banner = PostgresBanner()

In [2]:
def func(x):
    print("HELLO")
    print(x)
watch_banner.watch("test", func)

HELLO
{'topic': 'test', 'banner_timestamp': '20240907-214558005228', 'data': 'value'}
HELLO
{'topic': 'test', 'banner_timestamp': '20240907-214606494690', 'data': 'value'}
HELLO
{'topic': 'test', 'banner_timestamp': '20240907-214609243971', 'data': 'value'}
HELLO
{'topic': 'test', 'banner_timestamp': '20240907-214610068285', 'data': 'value'}
HELLO
{'topic': 'test', 'banner_timestamp': '20240907-214610557061', 'data': 'value'}
HELLO
{'topic': 'test', 'banner_timestamp': '20240907-214610917054', 'data': 'value'}
HELLO
{'topic': 'test', 'banner_timestamp': '20240907-214611252250', 'data': 'value'}
HELLO
{'topic': 'test', 'banner_timestamp': '20240907-214611598837', 'data': 'value'}
HELLO
{'topic': 'test', 'banner_timestamp': '20240907-214611878987', 'data': 'value'}
HELLO
{'topic': 'test', 'banner_timestamp': '20240907-214612151695', 'data': 'value'}
HELLO
{'topic': 'test', 'banner_timestamp': '20240907-214612370531', 'data': 'value'}


Exception in thread banners_watch_test:
Traceback (most recent call last):
  File "/opt/conda/envs/singleuser/lib/python3.11/threading.py", line 1045, in _bootstrap_inner
    self.run()
  File "/opt/conda/envs/singleuser/lib/python3.11/site-packages/ipykernel/ipkernel.py", line 761, in run_closure
    _threading_Thread_run(self)
  File "/opt/conda/envs/singleuser/lib/python3.11/threading.py", line 982, in run
    self._target(*self._args, **self._kwargs)
  File "/home/jovyan/work/banners/src/banners/postgres_banner.py", line 154, in _watch_thread
    callback(self._get_event_by_id(notify.payload))
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/jovyan/work/banners/src/banners/postgres_banner.py", line 81, in _get_event_by_id
    res = session.query(self.SQLBanner).where(self.SQLBanner.id == id)[0]
          ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~^^^
  File "/opt/conda/envs/singleuser/lib/python3.11/site-packages/sqlalchemy/orm/query.py", line 2531,

HELLO
{'topic': 'test', 'banner_timestamp': '20240907-214612586619', 'data': 'value'}


In [14]:
watch_banner.wave("test", {"data": "value"})

In [2]:
from typing import List
from typing import Optional
from sqlalchemy import ForeignKey
from sqlalchemy import String
from sqlalchemy.orm import DeclarativeBase
from sqlalchemy.orm import Mapped
from sqlalchemy.orm import mapped_column
from sqlalchemy.orm import relationship

class Base(DeclarativeBase):
    pass

class SQLBanner(Base):
    __tablename__ = "sql_banner2"

    id: Mapped[int] = mapped_column(primary_key=True)
    topic: Mapped[str]
    timestamp: Mapped[str]
    body: Mapped[Optional[str]]

    def __repr__(self) -> str:
        return f"User(topic={self.topic!r}, timestamp={self.timestamp!r}, body={self.body!r})"

In [16]:
banner = PostgresBanner()
# Base.metadata.create_all(banner._engine)
# banner.max_events_in_topic = 5

In [8]:
from sqlalchemy import inspect
inspector = inspect(banner._engine)
schemas = inspector.get_schema_names()

for schema in schemas:
    print("schema: %s" % schema)
    for table_name in inspector.get_table_names(schema=schema):
        print(table_name)
        # for column in inspector.get_columns(table_name, schema=schema):
            # print("Column: %s" % column)

schema: information_schema
sql_packages
sql_features
sql_implementation_info
sql_parts
sql_languages
sql_sizing
sql_sizing_profiles
schema: public
sqleventstore
sqlqueue
sql_banner


In [6]:
banner = PostgresBanner(table_name="sql_banner")
banner.SQLBanner.__table__.drop(banner._engine)

In [28]:
banner.wave("test", {"data": 6})

In [18]:
from sqlalchemy import select
from sqlalchemy.orm import Session

session = Session(banner._engine)

stmt = select(SQLBanner).where(SQLBanner.topic == "test")

for user in session.scalars(stmt):
    print(user)

User(topic='test', timestamp='20240907-201527614095', body='{"data": 6}')
User(topic='test', timestamp='20240907-201531658254', body='{"data": 6}')


In [23]:
session.query(SQLBanner) \
               .where(SQLBanner.id == 22)[0]

User(topic='test', timestamp='20240907-130944997853', body='{"data": 6}')

In [19]:
banner.recall_events("test", 10)
# banner.cleanup("test", 2)

[{'topic': 'test',
  'banner_timestamp': '20240907-214612370531',
  'data': 'value'},
 {'topic': 'test',
  'banner_timestamp': '20240907-214612586619',
  'data': 'value'}]